In [56]:
import pandas as pd
import numpy as np
from glob import glob
from random import randint, shuffle

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

In [36]:
# This a parameter to adjust as required
MAX_MEMORY = 20
RELOAD = True

FIXED_FEATURE_COLUMNS = ["innings", "bat_position", "bat_avg", "bat_sr", "bat_arm", "bowl_avg", "bowl_sr", "bowl_type", "seam_factor", "spin_factor"]
DYNAMIC_FEATURE_COLUMNS = ["match_balls", "inn_balls", "team_wkts", "team_lead", "bat_score", "bat_balls", "bowl_balls", "bowl_runs", "bowl_wkts"]
data = pd.DataFrame(columns = FIXED_FEATURE_COLUMNS + DYNAMIC_FEATURE_COLUMNS + ["seq"])
Y = []

# Load batter-bowler sequences
path = "../data/interim/batter_bowler_sequences"
all_files = glob(path + "/*.csv")
full_file = "../data/processed/bat_bowl_pred.csv"
Y_file = "../data/processed/bat_bowl_out.csv"

if RELOAD:
    for i,filename in enumerate(all_files):
        raw = pd.read_csv(filename)

        # Choose a random place in the sequence to predict from 
        seq_idx = randint(0, len(raw.index) - 1) if len(raw.index) > 1 else 0
        row = list(raw.iloc[seq_idx][FIXED_FEATURE_COLUMNS + DYNAMIC_FEATURE_COLUMNS])

        Y.append(raw.iloc[seq_idx]["outcome"])
        row.append(list(raw.iloc[:(seq_idx-1)]["outcome"]))

        data.loc[i] = row

    # This code takes quite a while to run
    
    # Remove na columns
    to_del = []
    for i,j in enumerate(data.index):
        if data.loc[j].isnull().values.any():
            data.drop(j, inplace=True)
            to_del.append(i)
            
    for d in reversed(to_del):
        del Y[d]
        
    data.to_csv(full_file, index=False)
    pd.DataFrame(Y).to_csv(Y_file, index=False)
else:
    data = pd.read_csv(full_file)

NameError: name 'DataFrame' is not defined

In [39]:
len(Y), len(data.index)
#pd.DataFrame(Y).to_csv(Y_file, index=False)

(50986, 50986)

50986

In [32]:
len(Y), len(data.index)


(24923, 24296)

In [19]:
#data.dropna(inplace=True)
data
#data.to_csv(full_file, index=False)

In [64]:
# Preprocess data
def one_hot(col):
    levels = list(set(col))
    encoded = np.zeros((len(col), len(levels)))
    
    for i,v in enumerate(col):
        encoded[i,levels.index(v)] = 1
    
    return encoded
    
def normalise(col):
    mmin = min(col)
    mmax = max(col)
    return [(float(i)-mmin)/(mmax-mmin) for i in col]
    

X = np.array((len(data.index, )))
CAT_COLS = ["innings", "bat_position", "bat_arm", "bowl_type"]
first = True
for c in data.columns[:-1]:
    new = one_hot(list(data[c])) if c in CAT_COLS else np.array(normalise(list(data[c])), ndmin = 2).transpose()
    if first:
        X = new
        first = False
    else:
        X = np.concatenate((X,new), axis=1)
        
# Pad sequences
padded = []
for seq in data["seq"]:
    row = [-1]*MAX_MEMORY
    
    for i,s in enumerate(reversed(seq)):
        if i > 20:
            break
        row[MAX_MEMORY-i-1] = 1 if s == "W" else 0
    
    padded.append(row)
padded = np.array(padded)
    
for i in range(0,MAX_MEMORY):
    X = np.concatenate((X, one_hot(padded[:,i])), axis=1)
 

# Encode output
Y = np.array([1 if y == "W" else 0 for y in Y])

In [44]:
X.shape
#Y.shape

(50986, 101)

In [66]:
# Split into training, testing, validation

shuff_ind = list(range(X.shape[0]))
shuffle(shuff_ind)
X = X[shuff_ind,:]
Y = Y[shuff_ind]


array([0, 0, 0, ..., 0, 0, 0])

In [45]:

model = Sequential()

model.add(Dense(318, input_dim=X.shape[1]))

# Recurrent layer
#model.add(LSTM(64, return_sequences=False, 
#               dropout=0.1, recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
model.fit(X,Y, epochs=150, batch_size=128)

Epoch 1/150
399/399 [==============================] - 1s 2ms/step - loss: 0.1968 - accuracy: 0.9436
Epoch 2/150
399/399 [==============================] - 1s 2ms/step - loss: 0.1962 - accuracy: 0.9436
Epoch 3/150
399/399 [==============================] - 1s 2ms/step - loss: 0.1955 - accuracy: 0.9436
Epoch 4/150
399/399 [==============================] - 1s 2ms/step - loss: 0.1952 - accuracy: 0.9436
Epoch 5/150
399/399 [==============================] - 1s 2ms/step - loss: 0.1950 - accuracy: 0.9436
Epoch 6/150
399/399 [==============================] - 1s 1ms/step - loss: 0.1956 - accuracy: 0.9436
Epoch 7/150
399/399 [==============================] - 1s 1ms/step - loss: 0.1951 - accuracy: 0.9436
Epoch 8/150
399/399 [==============================] - 1s 2ms/step - loss: 0.1949 - accuracy: 0.9436
Epoch 9/150
399/399 [==============================] - 1s 2ms/step - loss: 0.1951 - accuracy: 0.9436
Epoch 10/150
399/399 [==============================] - 1s 1ms/step - loss: 0.1949 - accura

In [47]:
# evaluate the keras model
_, accuracy = model.evaluate(X, Y)
print('Accuracy: %.2f' % (accuracy*100))

1594/1594 [==============================] - 1s 445us/step - loss: 0.1914 - accuracy: 0.9436
Accuracy: 94.36
